# Advent of code 2023

Solutions are my own, if any external source including hints have been used it shall be mentioned and linked.

## Part 1

--- Day 3: Gear Ratios ---

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?




In this example, the calibration values of these four lines are 12, 38, 15, and 77. Adding these together produces 142.

Consider your entire calibration document. What is the sum of all of the calibration values?

In [1]:
from dataclasses import dataclass
from typing import Generator
import math


LOCS = list[tuple[str,tuple[int]]]

@dataclass
class Number:
    """
    Represents a number with its locations in a puzzle.
    """
    locs: LOCS
    digit: int = None # Digit extracted from locations

    def get_digit(self):
        """Extracts the digit from locations."""
        return int("".join(num for num, _ in self.locs))

    def is_symbol(self, char: str) -> bool:
        """Checks if a character is a symbol."""
        if char == '.' or char.isdigit():
            return False
        return True

    def __post_init__(self):
        """Initializes the digit and coords after object creation."""
        self.digit = self.get_digit()
        self.coords = self.get_coords()

    def get_coords(self):
        return set(xy for _, xy in self.locs)

    def is_adjacent(self, puzzle:str)->bool:
        """
        Checks if a symbol is adjacent to any number location in the puzzle.
        Assumes origin (0,0) is the top leftmost of the string.
        """
        puzzle_lines = puzzle.splitlines()
        x_max, y_max = len(puzzle_lines[0]), len(puzzle_lines)

        valid_locs = set()
        for _, loc in self.locs:
            x, y = loc
            # Define adj. positions
            if x+1 < x_max: valid_locs.add((x+1, y)) # RIGHT,
            if x-1 >= 0: valid_locs.add((x-1,y)) # LEFT
            if y+1 < y_max: valid_locs.add((x, y+1)) # DOWN
            if y-1 >= 0: valid_locs.add((x, y-1)) # UP
            if x+1 < x_max and y-1 >= 0: valid_locs.add((x+1, y-1))# TOP RIGHT
            if x-1 >= 0 and y-1 >= 0: valid_locs.add((x-1, y-1))# TOP LEFT
            if x+1 < x_max and y+1 < y_max: valid_locs.add((x+1, y+1)) # BOTTOM RIGHT
            if x-1 >= 0  and y+1 < y_max: valid_locs.add((x-1, y+1)) # BOTTOM LEFT
        
        self.valid_locs = valid_locs
        # Check if any adjacent position contains a symbol
        return any(
            self.is_symbol(puzzle_lines[y][x])
            for x, y in self.valid_locs
        )

def parse_puzzle_numbers(puzzle:str)->Generator[Number, None, None]:
    """
    Parses the puzzle and lazily extracts a list of Number objects.

    """
    puzzle_lines = puzzle.splitlines()
    locs = list()
    numbers_list = list()
    in_number = False
    for y, row in enumerate(puzzle_lines):
        # account for line change
        if in_number:
            numbers_list.append(
                    Number(locs=locs)
                )
            locs = list() # purge locs
            in_number = False
        for x, char in enumerate(row):
            # start or continue number scan
            if char.isdecimal():
                locs.append((char,(x,y)))
                in_number = True
            # number scan finished
            elif len(locs) != 0:
                numbers_list.append(
                    Number(locs=locs)
                )
                locs = list() # purge locs
                in_number = False
    # will run out of space if number in bottom right
    if in_number:
        numbers_list.append(
                    Number(locs=locs))
    yield from numbers_list


def part1(numbers:list[Number], puzzle:str)->Generator[int, None, None]:
    """
    Lazily generates each numbers digit in puzzle
    if a symbol is adjacent
    """
    yield from (
        num.digit
        for num in numbers
        if num.is_adjacent(puzzle=puzzle)
    )

# account for line change corner cases and bottom vals
TEST2 = """467&.114..
...*......
..35|..633
6.....#...
617*......
.....+.58.
..592.....
.......755
....$.*..
.664...598"""


TEST = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

numbers_test = parse_puzzle_numbers(puzzle=TEST)
assert sum(part1(numbers=numbers_test, puzzle=TEST)) == 4361

In [2]:
numbers_test = parse_puzzle_numbers(puzzle=TEST)
list(numbers_test)

[Number(locs=[('4', (0, 0)), ('6', (1, 0)), ('7', (2, 0))], digit=467),
 Number(locs=[('1', (5, 0)), ('1', (6, 0)), ('4', (7, 0))], digit=114),
 Number(locs=[('3', (2, 2)), ('5', (3, 2))], digit=35),
 Number(locs=[('6', (6, 2)), ('3', (7, 2)), ('3', (8, 2))], digit=633),
 Number(locs=[('6', (0, 4)), ('1', (1, 4)), ('7', (2, 4))], digit=617),
 Number(locs=[('5', (7, 5)), ('8', (8, 5))], digit=58),
 Number(locs=[('5', (2, 6)), ('9', (3, 6)), ('2', (4, 6))], digit=592),
 Number(locs=[('7', (6, 7)), ('5', (7, 7)), ('5', (8, 7))], digit=755),
 Number(locs=[('6', (1, 9)), ('6', (2, 9)), ('4', (3, 9))], digit=664),
 Number(locs=[('5', (5, 9)), ('9', (6, 9)), ('8', (7, 9))], digit=598)]

## Part 2


The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?


In [3]:
@dataclass
class Gear:
    """
    Represents a Gear with its locations in a puzzle.
    """
    loc: tuple[int]
    

    def is_gear(self, puzzle:str, numbers:list[Number]) -> True:
        """Checks if a gear is adjacent to two part numbers
        computes the gear ratio and number it contains"""
        
        # set the gear to False initially
        self.is_gear= False
        # compute valid location around '*'
        self._valid_locs(puzzle=puzzle)
        
        part_numbers = list()
        for num in numbers:
            if num.is_adjacent(puzzle=puzzle):
                # define coords. intersection
                # print(num)
                if self.valid_locs & num.coords:
                    part_numbers.append(num)
        if len(part_numbers) == 2: # exactly two part numbers
            self.part_numbers = part_numbers
            self.is_gear = True
            self.gear_ratio = math.prod(n.digit for n in self.part_numbers)
            return True
        return False

    def _valid_locs(self, puzzle:str)->None:
        """
        creates valid coordinates of a Gear given puzzle
        Assumes origin (0,0) is the top leftmost of the string.
        """
        puzzle_lines = puzzle.splitlines()
        x_max, y_max = len(puzzle_lines[0]), len(puzzle_lines)

        valid_locs = set()
        x, y = self.loc
        # Define adj. positions
        if x+1 < x_max: valid_locs.add((x+1, y)) # RIGHT,
        if x-1 >= 0: valid_locs.add((x-1,y)) # LEFT
        if y+1 < y_max: valid_locs.add((x, y+1)) # DOWN
        if y-1 >= 0: valid_locs.add((x, y-1)) # UP
        if x+1 < x_max and y-1 >= 0: valid_locs.add((x+1, y-1))# TOP RIGHT
        if x-1 >= 0 and y-1 >= 0: valid_locs.add((x-1, y-1))# TOP LEFT
        if x+1 < x_max and y+1 < y_max: valid_locs.add((x+1, y+1)) # BOTTOM RIGHT
        if x-1 >= 0  and y+1 < y_max: valid_locs.add((x-1, y+1)) # BOTTOM LEFT
    
        self.valid_locs = valid_locs



def parse_puzzle_gears(puzzle:str)->Generator[Gear, None, None]:
    """
    Parses the puzzle and lazily extracts a list of Number objects.
    """
    puzzle_lines = puzzle.splitlines()
    for y, row in enumerate(puzzle_lines):
        # account for line change
        for x, char in enumerate(row):
            # start or continue number scan
            if char == '*':
                # print(char, (x, y))
                yield Gear(loc=(x,y))
       
        
def part2(puzzle:str, numbers:list[Number])->int:
    yield from (g.gear_ratio 
                for g in parse_puzzle_gears(puzzle=puzzle) 
                if g.is_gear(puzzle=puzzle, numbers=numbers)
                )
numbers = list(parse_puzzle_numbers(puzzle=TEST))
assert sum(part2(puzzle=TEST, numbers=numbers)) == 467835

## Solutions

In [4]:
with open("puzzle_input/day03.txt") as file:
    puzzle = file.read()

numbers = list(parse_puzzle_numbers(puzzle=puzzle))
print("part1", sum(part1(numbers=numbers, puzzle=puzzle)))
print("part2", sum(part2(puzzle=puzzle, numbers=numbers)))

part1 527369


part2 73074886
